## SQLite Questions 10-13


In [145]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql


In [21]:
country_club = "sqlite_db_pythonsqlite.db"
connection = sql.connect(country_club)


In [22]:
query = '''SELECT * From FACILITIES'''

In [23]:
df = pd.read_sql_query(query,connection)

In [24]:
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [25]:
query = '''SELECT * FROM BOOKINGS '''

In [26]:
df = pd.read_sql_query(query,connection)

In [27]:
df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [28]:
query = '''SELECT f.name, 
SUM(CASE WHEN b.memid = 0 
THEN f.guestcost * b.slots
ELSE f.membercost * b.slots
END) AS revenue
FROM `Facilities` AS f 
INNER JOIN `Bookings` AS b
ON f.facid = b.facid
GROUP BY 1
HAVING revenue < 1000
ORDER BY revenue

'''

In [29]:
df = pd.read_sql_query(query,connection)

In [30]:
print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [136]:
query = '''SELECT (m.firstname || ' ' || m.surname) AS Member, (m1.firstname || ' ' || m1.surname) AS Recommender
FROM `Members` AS m
LEFT JOIN `Members` AS m1 
ON m.recommendedby = m1.memid
WHERE m1.memid != 0
ORDER BY m.surname ASC'''

In [137]:
df = pd.read_sql_query(query,connection)

In [138]:
print(df)

                     Member        Recommender
0            Florence Bader    Ponder Stibbons
1                Anne Baker    Ponder Stibbons
2             Timothy Baker     Jemima Farrell
3                Tim Boothe         Tim Rownam
4            Gerald Butters       Darren Smith
5               Joan Coplin      Timothy Baker
6             Erica Crumpet        Tracy Smith
7                Nancy Dare    Janice Joplette
8           Matthew Genting     Gerald Butters
9                 John Hunt  Millicent Purview
10              David Jones    Janice Joplette
11            Douglas Jones        David Jones
12          Janice Joplette       Darren Smith
13           Anna Mackenzie       Darren Smith
14             Charles Owen       Darren Smith
15             David Pinker     Jemima Farrell
16        Millicent Purview        Tracy Smith
17         Henrietta Rumney    Matthew Genting
18         Ramnaresh Sarwin     Florence Bader
19               Jack Smith       Darren Smith
20          P

Q12: Find the facilities with their usage by member, but not guests

In [139]:
query = '''SELECT f.name AS facility, mem.Memberusage AS MEMBER_USAGE
FROM (SELECT facid, name FROM `Facilities`) AS f
INNER JOIN 
(SELECT facid, COUNT(*) AS Memberusage FROM `Bookings`
WHERE memid != 0 
GROUP BY facid) AS mem
ON f.facid = mem.facid'''

In [140]:
df = pd.read_sql_query(query,connection)

In [141]:
print(df)

          facility  MEMBER_USAGE
0   Tennis Court 1           308
1   Tennis Court 2           276
2  Badminton Court           344
3     Table Tennis           385
4   Massage Room 1           421
5   Massage Room 2            27
6     Squash Court           195
7    Snooker Table           421
8       Pool Table           783


Q13: Find the facilities usage by month, but not guests

In [142]:
query = '''SELECT f.name AS facility, mem.Monthly AS Monthly_Usage, mem.Memberusage AS MEMBER_USAGE
FROM (SELECT facid, name
FROM  `Facilities`
) AS f
INNER JOIN (
SELECT facid, strftime('%Y-%m', starttime) AS Monthly, COUNT( * ) AS Memberusage
FROM  `Bookings` 
WHERE memid !=0
GROUP BY facid, Monthly
) AS mem ON f.facid = mem.facid'''

In [143]:
df = pd.read_sql_query(query,connection)

In [144]:
print(df)

           facility Monthly_Usage  MEMBER_USAGE
0    Tennis Court 1       2012-07            65
1    Tennis Court 1       2012-08           111
2    Tennis Court 1       2012-09           132
3    Tennis Court 2       2012-07            41
4    Tennis Court 2       2012-08           109
5    Tennis Court 2       2012-09           126
6   Badminton Court       2012-07            51
7   Badminton Court       2012-08           132
8   Badminton Court       2012-09           161
9      Table Tennis       2012-07            48
10     Table Tennis       2012-08           143
11     Table Tennis       2012-09           194
12   Massage Room 1       2012-07            77
13   Massage Room 1       2012-08           153
14   Massage Room 1       2012-09           191
15   Massage Room 2       2012-07             4
16   Massage Room 2       2012-08             9
17   Massage Room 2       2012-09            14
18     Squash Court       2012-07            23
19     Squash Court       2012-08       